# This script implements a simple baseline-CNN model for the Music Genre Classification task.

In [1]:
# System/zip-handling imports
import os, sys
import zipfile

# Imports tensorflow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# Imports image handling
import cv2
import numpy as np
import skimage

# For generating training and test data
import random

# Save training progress
import csv
from datetime import datetime
from shutil import copyfile  # Making copy of this file instance (including param settings used)


W0111 16:29:24.988758 139649217668864 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0111 16:29:25.020673 139649217668864 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0111 16:29:25.041310 139649217668864 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.


# Get sorted list of training file names:

In [2]:
# Get access to zip-archive
archive = zipfile.ZipFile('../data/spectrograms.zip', 'r')
imgdata = archive.read('spectrograms/spectrogram_0000.png')

files = sorted([f for f in archive.namelist()[1:] if f.startswith('spectrograms/') and f.endswith('.png')])

print(files)

['spectrograms/spectrogram_0000.png', 'spectrograms/spectrogram_0001.png', 'spectrograms/spectrogram_0002.png', 'spectrograms/spectrogram_0003.png', 'spectrograms/spectrogram_0004.png', 'spectrograms/spectrogram_0005.png', 'spectrograms/spectrogram_0006.png', 'spectrograms/spectrogram_0007.png', 'spectrograms/spectrogram_0008.png', 'spectrograms/spectrogram_0009.png', 'spectrograms/spectrogram_0010.png', 'spectrograms/spectrogram_0011.png', 'spectrograms/spectrogram_0012.png', 'spectrograms/spectrogram_0013.png', 'spectrograms/spectrogram_0014.png', 'spectrograms/spectrogram_0015.png', 'spectrograms/spectrogram_0016.png', 'spectrograms/spectrogram_0017.png', 'spectrograms/spectrogram_0018.png', 'spectrograms/spectrogram_0019.png', 'spectrograms/spectrogram_0020.png', 'spectrograms/spectrogram_0021.png', 'spectrograms/spectrogram_0022.png', 'spectrograms/spectrogram_0023.png', 'spectrograms/spectrogram_0024.png', 'spectrograms/spectrogram_0025.png', 'spectrograms/spectrogram_0026.png', 

# JFF: Get a feeling for the nature of the training and evaluation data:

In [3]:
# Interpret image-data as image
#image = cv2.imdecode(np.frombuffer(imgdata, dtype=np.uint8), 1)

# Print some specs...
#print('Type of image:')
#print(type(image))
#print('Dimension of a single image file:')
#print(image.shape)

# Show image visually (press any key in opening window in order to proceed in code...)
#cv2.imshow("Image", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Read in both training and testing data from zip archive:

In [4]:
data_set = []
# Data storage
combined_data = np.empty([1, 128, 128])

# Read in images & store processed instances
for f_name in files:
    # Get image data from zip file
    zip_img_data = archive.read(f_name)
    image = cv2.imdecode(np.frombuffer(zip_img_data, dtype=np.uint8), 1)
    
    # Normalize image's colors to range [0, 1]
    image = image / 255.0

    #cv2.imshow("Normalized Image", image)
    #cv2.waitKey(0)

    # Grayscale image
    gray_image = skimage.color.rgb2gray(image)

    #cv2.imshow("Grayscale Image", gray_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # Store grayscaled image
    combined_data = np.append(combined_data, [gray_image], axis=0)
    
# Remove initial, empty datapoint
combined_data = combined_data[1:, :, :]

print('Done reading in... Shape of data array:')
print(combined_data.shape)
print('Done.')

Done reading in... Shape of data array:
(1000, 128, 128)
Done.


# Read in labels:

In [5]:
labels_path = '../data/labels.txt'

combined_labels = np.empty([1])

with open(labels_path, 'r') as file:
    for line in file:
        combined_labels = np.append(combined_labels, [int(line)])

# Remove initial, empty datapoint
combined_labels = combined_labels[1:] 
print(combined_labels)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.

# Divide data into train and test data:

------------------------------------------------------
Training data will be contained in:    training_data
Tetsing  data will be contained in:    testing_data

Training labels will be contained in:  training_labels
Tetsing  labels will be contained in:  testing_labels

In [6]:
# Get set of test-indices which indicates the training data points that have to be reserved for training
percentage_test_data = 0.2
population = range(len(combined_labels))
nr_samples = int(percentage_test_data * len(combined_labels))

test_indices = random.sample(population, nr_samples)
test_indices = sorted(test_indices)

print('Nr. test indices: ' + str(len(test_indices)))
print('Test indices: ' + str(test_indices))


# Split data into training- and test data, respectively - Preparation: Create empty arrays in which to later insert data
test_len = len(test_indices)
train_len = len(combined_labels)-len(test_indices)
training_data, training_labels = np.empty([train_len, 128, 128]), np.empty([train_len])
testing_data, testing_labels = np.empty([test_len, 128, 128]), np.empty([test_len])

test_idx_list_idx = 0
i = 0

# Iterate through all data and assign each data point either to training data or testing data
for data_idx in range(len(combined_labels)):

    if test_idx_list_idx < nr_samples and data_idx == test_indices[test_idx_list_idx]:
        testing_data[test_idx_list_idx, :, :] = combined_data[data_idx, :, :]
        testing_labels[test_idx_list_idx] = combined_labels[data_idx]
        test_idx_list_idx += 1
        
    else:
        
        training_data[i, :, :] = combined_data[data_idx, :, :]
        training_labels[i] = combined_labels[data_idx]
        i += 1
        

training_data = training_data.reshape([len(training_labels), 128, 128, 1])

testing_data = testing_data.reshape([len(testing_labels), 128, 128, 1])
        
print('Final:')
print(training_data.shape)
print(training_labels.shape)
print(testing_data.shape)
print(testing_labels.shape)


Nr. test indices: 200
Test indices: [6, 7, 8, 13, 14, 20, 25, 31, 40, 42, 45, 53, 57, 59, 64, 76, 99, 101, 102, 108, 114, 116, 119, 128, 132, 133, 135, 140, 146, 148, 154, 158, 159, 169, 178, 183, 184, 190, 204, 206, 214, 225, 236, 237, 239, 245, 246, 257, 263, 266, 279, 282, 284, 288, 293, 296, 297, 299, 311, 322, 330, 338, 345, 351, 358, 363, 366, 372, 377, 378, 382, 384, 387, 391, 401, 403, 405, 412, 415, 416, 425, 437, 447, 453, 459, 484, 488, 490, 504, 510, 511, 521, 530, 533, 534, 537, 546, 547, 548, 552, 555, 559, 562, 564, 565, 572, 575, 577, 580, 584, 585, 588, 592, 597, 608, 611, 612, 614, 615, 621, 630, 632, 636, 647, 648, 654, 655, 657, 664, 666, 668, 681, 683, 703, 704, 709, 712, 722, 723, 728, 737, 748, 749, 750, 754, 758, 774, 776, 783, 791, 805, 820, 830, 833, 835, 850, 853, 861, 871, 878, 879, 884, 885, 888, 893, 895, 897, 905, 908, 909, 911, 912, 913, 914, 917, 919, 925, 935, 936, 939, 941, 942, 954, 958, 964, 967, 970, 972, 976, 980, 981, 982, 984, 985, 986, 988, 990

In [7]:
#Set up folder for data gathering during training process

In [8]:
# Set up folder for data gathering during training process
now = datetime.now()
TIME_STAMP = now.strftime("_%Y_%d_%m__%H_%M_%S__%f")
MODEL_ID = 'Model_' + TIME_STAMP + '/'
training_path = '../Trained_Models/CNN_Models/'
path = training_path + MODEL_ID + '/'

if not os.path.exists(path):
    os.makedirs(path)
else:
    path = None
    raise Exception('PATH EXISTS!')

# Set up the CNN architecture:

In [9]:
%%writefile $path/model_settings.text 
# Line above: Save model settings to file for reproducability - Run once with command above and once without. 
# First, it saves cell's content, but doesn't run the cell, afterwards, it's running the cell

# Reset tf sessions
tf.keras.backend.clear_session()  # Destroys the current TF graph and creates a new one.

dimensions = 128
classes = 10

# Set up model architecture in terms of its layers
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(dimensions, dimensions, 1),
                                                       kernel_regularizer=regularizers.l2(0.01)))

model.add(layers.Dropout(0.05))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.05))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.05)))

model.add(layers.Dropout(0.1))

model.add(layers.Dense(classes, activation='softmax'))

# Note on regularizer(s), copied from https://www.tensorflow.org/tutorials/keras/overfit_and_underfit:
# l2(0.001) means that every coefficient in the weight matrix of the layer will add 0.001 * weight_coefficient_value**2
# to the total loss of the network.

# Print summary
model.summary()

# Compile model & make some design choices
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001, #0.0
                                           beta_1=0.9,
                                           beta_2=0.999,
                                           epsilon=1e-07,
                                           amsgrad=False,
                                           name='Adam'
                                           ),
              loss='sparse_categorical_crossentropy',  # Capable of working with regularization
              metrics=['accuracy', 'sparse_categorical_crossentropy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0

# Execute training:

Start training process:
    Run X times Y tensorflow-epochs and save a model as checkpoint after any Y epochs. 
    FIXME: Bit hacky solution, yet, but can be prettyfied.
    
IMPORTANT: 'accuracy'     == accuracy achieved during training on training data;  * the UN-important measure
           'val_accuracy' == accuracy achieved on TEST data AFTER training epoch; * the important measure

In [10]:
# Definition of callbacks adjusted from https://www.tensorflow.org/guide/keras/train_and_evaluate

# Define callbacks
early_stopping_callback = EarlyStopping(
        monitor='val_accuracy',    # Stop training when `val_loss` is no longer improving
        min_delta=0,               # "no longer improving" being defined as "no better than 0|5e-1 less"
        patience=2,                # "no longer improving" being further defined as "for at least 2 epochs"
        verbose=0)                 # Quantity of printed output

model_saving_callback = ModelCheckpoint(
        filepath=path+'cnn_model.h5',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_accuracy',
        # mode: one of {auto, min, max}. If `save_best_only=True`, the decision to
        # overwrite the current save file is made based on either the maximization
        # or the minimization of the monitored quantity. For `val_acc`, this
        # should be `max`, for `val_loss` this should be `min`, etc. In `auto`
        # mode, the direction is automatically inferred from the name of the
        # monitored quantity.
        verbose=0)

# Join list of required callbacks
callbacks = [model_saving_callback] #early_stopping_callback

# Set number of desired epochs (mind the early-stopping!)
epochs = 50

# Perform x epochs of training
history = model.fit(training_data, training_labels,
                    epochs=epochs,
                    validation_data=(testing_data, testing_labels),
                    callbacks=callbacks, verbose=1)

# Save the entire model as a final model to a HDF5 file.
name = 'final_model'
model.save(path+name+'.h5')

# Record training progress
with open(path+'training_progress.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["epoch", "loss", "accuracy", "val_loss", "val_accuracy", "sparse_categorical_crossentropy"])
    for line in range(len(history.history['loss'])): 
        epoch = str(line+1)
        writer.writerow([epoch,
                         history.history["loss"][line], 
                         history.history["accuracy"][line], 
                         history.history["val_loss"][line], 
                         history.history["val_accuracy"][line],
                         history.history["sparse_categorical_crossentropy"][line]
                         ])
    file.close()

print('Done.')

Train on 800 samples, validate on 200 samples
Epoch 1/50


W0111 16:30:00.569128 139649217668864 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


800/800 [==============================] - 14s 18ms/sample - loss: 3.8014 - accuracy: 0.1025 - sparse_categorical_crossentropy: 2.3129 - val_loss: 2.6628 - val_accuracy: 0.1200 - val_sparse_categorical_crossentropy: 2.3010
Epoch 2/50
800/800 [==============================] - 15s 18ms/sample - loss: 2.5088 - accuracy: 0.1412 - sparse_categorical_crossentropy: 2.2946 - val_loss: 2.4083 - val_accuracy: 0.1750 - val_sparse_categorical_crossentropy: 2.2876
Epoch 3/50
800/800 [==============================] - 13s 17ms/sample - loss: 2.3743 - accuracy: 0.1312 - sparse_categorical_crossentropy: 2.2817 - val_loss: 2.3620 - val_accuracy: 0.0850 - val_sparse_categorical_crossentropy: 2.2898
Epoch 4/50
800/800 [==============================] - 15s 19ms/sample - loss: 2.3268 - accuracy: 0.1562 - sparse_categorical_crossentropy: 2.2458 - val_loss: 2.3439 - val_accuracy: 0.1250 - val_sparse_categorical_crossentropy: 2.2581
Epoch 5/50
800/800 [==============================] - 16s 20ms/sample - los

Epoch 36/50
800/800 [==============================] - 13s 16ms/sample - loss: 1.5745 - accuracy: 0.5813 - sparse_categorical_crossentropy: 1.2191 - val_loss: 1.9918 - val_accuracy: 0.4600 - val_sparse_categorical_crossentropy: 1.6339
Epoch 37/50
800/800 [==============================] - 12s 16ms/sample - loss: 1.5742 - accuracy: 0.5838 - sparse_categorical_crossentropy: 1.2136 - val_loss: 2.0551 - val_accuracy: 0.4050 - val_sparse_categorical_crossentropy: 1.6893
Epoch 38/50
800/800 [==============================] - 12s 16ms/sample - loss: 1.5113 - accuracy: 0.6225 - sparse_categorical_crossentropy: 1.1386 - val_loss: 2.0034 - val_accuracy: 0.4300 - val_sparse_categorical_crossentropy: 1.6280
Epoch 39/50
800/800 [==============================] - 12s 16ms/sample - loss: 1.5468 - accuracy: 0.6000 - sparse_categorical_crossentropy: 1.1702 - val_loss: 2.0735 - val_accuracy: 0.3900 - val_sparse_categorical_crossentropy: 1.6929
Epoch 40/50
800/800 [==============================] - 13s 1